In [1]:
import sys
import os
sys.path.append(os.path.expanduser('~/workspace/tacotron/'))

from util import audio
from hparams import hparams
from lib import sigproc as sp
import librosa
import numpy as np
from tacotron.pml_synthesizer import Configuration, PMLSynthesizer
from tqdm import tqdm_notebook as tqdm

In [2]:
training_data_dir = '/media/josh/Store/tacotron-data/gran-lj-training'
metadata_filename = os.path.join(training_data_dir, 'test.txt')

cfg = Configuration(16000, 86)
synth = PMLSynthesizer(cfg)
# synth.load(checkpoint_path, hparams, gta=gta, model_name=args.variant)

with open(metadata_filename, encoding='utf-8') as f:
    metadata = [line.strip().split('|') for line in f]
    hours = sum((int(x[2]) for x in metadata)) * hparams.frame_shift_ms / (3600 * 1000)
    print('Loaded metadata for %d examples (%.2f hours)' % (len(metadata), hours))
    
pml_features = [m[3] for m in metadata]
texts = [m[5] for m in metadata]
wav_files = [m[6] for m in metadata]

Loaded metadata for 150 examples (0.68 hours)


In [3]:
gt_wavs = []
analysis_synthesis_wavs = []

for wav_file in tqdm(wav_files):
    gt_wav_path = os.path.join(training_data_dir, 'wavs', wav_file)
    gt_wav = audio.load_wav(gt_wav_path)
    gt_wavs.append(gt_wav)
    
    as_wav_path = os.path.join(training_data_dir, 'analysis_synthesis', wav_file)
    as_wav = audio.load_wav(as_wav_path)
    analysis_synthesis_wavs.append(as_wav)

In [4]:
from lib.pulsemodel import analysis

# def analysis(wav, fs, f0s=None, f0_min=60, f0_max=600, f0estimator='REAPER',
#              shift=0.005,    # Usually 5ms
#              dftlen=4096,    # You can adapt this one according to your pipeline
#              verbose=1):

#     if verbose>0: print('PML Analysis (dur={:.3f}s, fs={}Hz, f0 in [{},{}]Hz, shift={}s, dftlen={})'.format(len(wav)/float(fs), fs, f0_min, f0_max, shift, dftlen))

#     f0s = analysis_f0postproc(wav, fs, f0s, f0_min=f0_min, f0_max=f0_max, shift=shift, f0estimator=f0estimator, verbose=verbose)

#     SPEC = analysis_spec(wav, fs, f0s, shift=shift, dftlen=dftlen, verbose=verbose)

#     PDD = analysis_pdd(wav, fs, f0s, dftlen=dftlen, verbose=verbose)

#     NM = analysis_nm(wav, fs, f0s, PDD, verbose=verbose)

#     if verbose>2:
#         plot_features(wav=wav, fs=fs, f0s=f0s, SPEC=SPEC, PDD=PDD, NM=NM) # pragma: no cover

#     return f0s, SPEC, PDD, NM

def pml_metric(first_set, second_set, sr_1=16000, sr_2=16000, base_sr=16000, debug=False):
    error = 0.0
    wav_set_size = len(first_set)

    for i in range(wav_set_size):
        wav_1 = first_set[i]
        wav_2 = second_set[i]
        
        if debug:
            print('Sample Rates:', sr_1, sr_2, base_sr)
        
        if sr_1 != base_sr:
            wav_1 = librosa.core.resample(wav_1, orig_sr=sr_1, target_sr=base_sr)
        
        if sr_2 != base_sr:
            wav_2 = librosa.core.resample(wav_2, orig_sr=sr_2, target_sr=base_sr)
        
        cutoff = min(len(wav_1), len(wav_2))
        f0s_1, _, _, _ = analysis(np.array(wav_1[:cutoff], dtype=np.int16), base_sr)
        f0s_2, _, _, _ = analysis(np.array(wav_2[:cutoff], dtype=np.int16), base_sr)

        error += np.mean(np.square(f0s_1 - f0s_2))

    error = np.sqrt(error / wav_set_size)
    return error

In [6]:
pml_metric(gt_wavs, analysis_synthesis_wavs, sr_1=22050, sr_2=22050, base_sr=22050)

PML Analysis (dur=6.776s, fs=22050Hz, f0 in [60,600]Hz, shift=0.005s, dftlen=4096)


FileNotFoundError: [Errno 2] No such file or directory: '/dev/shm/josh/sigproc.pid2433.6b6278a5-67e1-41c3-bd7c-3748a64fa2db.interface-reaper-out.f0'